<a href="https://colab.research.google.com/github/DineshSiddhartha/ML_Assignment3/blob/main/Question1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from pprint import pprint

In [ ]:
import pandas as pd
import requests
import re
from pprint import pprint

# Function to fetch and clean data from the URL
def fetch_and_clean_data(url):
    response = requests.get(url)
    response.raise_for_status()
    lines = response.text.split('\n')

    cleaned_lines = []
    for line in lines:
        cleaned_line = line.strip().lower()
        if cleaned_line:
            cleaned_line = re.sub(r'[^a-zA-Z0-9 \.]', '', cleaned_line)
            cleaned_lines.append(cleaned_line)

    return cleaned_lines

url = 'https://www.gutenberg.org/files/1661/1661-0.txt'
cleaned_lines = fetch_and_clean_data(url)

text = ' '.join(cleaned_lines)

words = text.split()

unique_words = sorted(set(words))
stoi = {s: i + 1 for i, s in enumerate(unique_words)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}

pprint(itos)


In [ ]:
import torch

block_size = 5
X, Y = [], []

# Creating sequences for next-word prediction
for sentence in cleaned_lines:
    word_list = sentence.split()
    context = [0] * block_size

    for word in word_list + ['.']:
        ix = stoi.get(word, stoi['.'])
        X.append(context.copy())
        Y.append(ix)

        print(' '.join(itos[i] for i in context), '--->', itos[ix])

        context = context[1:] + [ix]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.tensor(X).to(device)
Y = torch.tensor(Y).to(device)

print(f'Shape of X: {X.shape}, Shape of Y: {Y.shape}')

In [ ]:
emb_dim = 32
emb = torch.nn.Embedding(len(stoi), emb_dim)
import torch
import torch.nn as nn

class NextWord(nn.Module):
    def __init__(self, block_size, vocab_size, emb_dim, hidden_size=1024):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lin1 = nn.Linear(block_size * emb_dim, hidden_size)
        self.lin2 = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.shape[0], -1)
        x = torch.relu(self.lin1(x))
        x = self.lin2(x)
        return x

model = NextWord(block_size, len(stoi), emb_dim, 1024).to(device)
model = torch.compile(model)


In [ ]:
# Generate names from untrained model


g = torch.Generator()
g.manual_seed(4000002)

def generate_name(model, itos, stoi, block_size, max_len=10):
    context = [0] * block_size
    name = ''
    for i in range(max_len):
        x = torch.tensor(context).view(1, -1).to(device)
        y_pred = model(x)
        ix = torch.distributions.categorical.Categorical(logits=y_pred).sample().item()
        word = itos[ix]

        if word == '.':
            break

        name += (' ' + word) if name else word
        context = context[1:] + [ix]
    return name

for i in range(10):
    print(generate_name(model, itos, stoi, block_size))


u.s.a. completely. inquest. dorans wood thumped welllit nightbird thieves 1.
wifeyou happening jezail waterpolice interest. cuttings. actor smokeless host gloss
sharpeyed moran. thanks fund reptiles thoroughfare. children temper. blotches. resided
tattoo 1.f.2. mouth paused intellectual hungrily known domain damp george
research deceive advertisementhow hercules. villain drivingrod another hospitality presuming horrors
inexplicable whitewashed glad prompt faith dropping satisfied expostulating holland. moonlight
pacific smiling. meditation lateral compress prankupon relations hearty eyebrows. ones.
hayling flaming landingplaces serves unlike pompous whom mean lonely sense
lets cashier property. frockcoat escape lip farther donations independent described
uproar. walked moran backi repair buttend collar indicating geniality crossed


In [ ]:
import torch

model_load_path = '/content/model_emb_32_block_5_relu.pth'
model.load_state_dict(torch.load(model_load_path, map_location='cpu', weights_only=True))
model.train()
import torch._dynamo
torch._dynamo.config.suppress_errors = True

loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.AdamW(model.parameters(), lr=0.01)

batch_size = 4000
save_every = 50
additional_epochs = 650
start_epoch = 0

elapsed_time = []

for epoch in range(start_epoch, start_epoch + additional_epochs):
    for i in range(0, X.shape[0], batch_size):
        x = X[i:i + batch_size]
        y = Y[i:i + batch_size]

        y_pred = model(x)
        loss = loss_fn(y_pred.view(-1, len(stoi)), y.view(-1))
        loss.backward()
        opt.step()
        opt.zero_grad()

    if epoch % save_every == 0:
        torch.save(model.state_dict(), model_load_path)
    print(f'Epoch: {epoch}, Loss: {loss.item():.4f}')


In [ ]:
model_save_path = '/content/model_emb_32_block_5_relu.pth'
torch.save(model.state_dict(), model_save_path)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def plot_embeddings(embedding_layer, itos):
    embeddings = embedding_layer.weight.data.cpu().numpy()
    words = [itos.get(i) for i in range(len(embeddings))]

    tsne = TSNE(n_components=2, random_state=42)
    reduced_embeddings = tsne.fit_transform(embeddings)

    plt.figure(figsize=(12, 12))
    plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1])

    for i, word in enumerate(words):
        plt.annotate(word, (reduced_embeddings[i, 0], reduced_embeddings[i, 1]), fontsize=8, alpha=0.5)

    plt.title('t-SNE Visualization of Word Embeddings')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.grid(True)
    plt.show()

plot_embeddings(model.emb, itos)


In [ ]:
model_load_path = '/content/drive/MyDrive/model_emb_32_block_5_relu.pth'
model.load_state_dict(torch.load(model_load_path, map_location='cpu', weights_only=True))
plot_embeddings(model.emb, itos)

In [ ]:
import torch

block_size = 10
X, Y = [], []

for sentence in cleaned_lines:
    word_list = sentence.split()
    context = [0] * block_size

    for word in word_list + ['.']:
        ix = stoi.get(word, stoi['.'])
        X.append(context.copy())
        Y.append(ix)
        context = context[1:] + [ix]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.tensor(X).to(device)
Y = torch.tensor(Y).to(device)

print(f'Shape of X: {X.shape}, Shape of Y: {Y.shape}')

emb_dim = 32
emb = torch.nn.Embedding(len(stoi), emb_dim)
import torch
import torch.nn as nn

class NextWord(nn.Module):
    def __init__(self, block_size, vocab_size, emb_dim, hidden_size=1024):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lin1 = nn.Linear(block_size * emb_dim, hidden_size)
        self.lin2 = nn.Linear(hidden_size, vocab_size)
    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.shape[0], -1)
        x = torch.relu(self.lin1(x))
        x = self.lin2(x)
        return x

model = NextWord(block_size, len(stoi), emb_dim, 1024).to(device)
model = torch.compile(model)
model_load_path = '/content/drive/MyDrive/model_emb_32_block_10_relu.pth'
model.load_state_dict(torch.load(model_load_path, map_location='cpu', weights_only=True))
plot_embeddings(model.emb, itos)

In [ ]:
import torch

block_size = 15
X, Y = [], []

for sentence in cleaned_lines:
    word_list = sentence.split()
    context = [0] * block_size

    for word in word_list + ['.']:
        ix = stoi.get(word, stoi['.'])
        X.append(context.copy())
        Y.append(ix)
        context = context[1:] + [ix]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.tensor(X).to(device)
Y = torch.tensor(Y).to(device)

print(f'Shape of X: {X.shape}, Shape of Y: {Y.shape}')

emb_dim = 32
emb = torch.nn.Embedding(len(stoi), emb_dim)
import torch
import torch.nn as nn

class NextWord(nn.Module):
    def __init__(self, block_size, vocab_size, emb_dim, hidden_size=1024):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lin1 = nn.Linear(block_size * emb_dim, hidden_size)
        self.lin2 = nn.Linear(hidden_size, vocab_size)
    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.shape[0], -1)
        x = torch.relu(self.lin1(x))
        x = self.lin2(x)
        return x

model = NextWord(block_size, len(stoi), emb_dim, 1024).to(device)
model = torch.compile(model)
model_load_path = '/content/drive/MyDrive/model_emb_32_block_15_relu.pth'
model.load_state_dict(torch.load(model_load_path, map_location='cpu', weights_only=True))
plot_embeddings(model.emb, itos)

In [ ]:
import torch

block_size = 10
X, Y = [], []

for sentence in cleaned_lines:
    word_list = sentence.split()
    context = [0] * block_size

    for word in word_list + ['.']:
        ix = stoi.get(word, stoi['.'])
        X.append(context.copy())
        Y.append(ix)
        context = context[1:] + [ix]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.tensor(X).to(device)
Y = torch.tensor(Y).to(device)

print(f'Shape of X: {X.shape}, Shape of Y: {Y.shape}')

emb_dim = 32
emb = torch.nn.Embedding(len(stoi), emb_dim)
import torch
import torch.nn as nn

class NextWord(nn.Module):
    def __init__(self, block_size, vocab_size, emb_dim, hidden_size=1024):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lin1 = nn.Linear(block_size * emb_dim, hidden_size)
        self.lin2 = nn.Linear(hidden_size, vocab_size)
    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.shape[0], -1)
        x = torch.tanh(self.lin1(x))
        x = self.lin2(x)
        return x

model = NextWord(block_size, len(stoi), emb_dim, 1024).to(device)
model = torch.compile(model)
model_load_path = '/content/drive/MyDrive/model_emb_32_block_5_tanh.pth'
model.load_state_dict(torch.load(model_load_path, map_location='cpu', weights_only=True))
plot_embeddings(model.emb, itos)

In [ ]:
import torch

block_size = 10
X, Y = [], []

for sentence in cleaned_lines:
    word_list = sentence.split()
    context = [0] * block_size

    for word in word_list + ['.']:
        ix = stoi.get(word, stoi['.'])
        X.append(context.copy())
        Y.append(ix)
        context = context[1:] + [ix]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.tensor(X).to(device)
Y = torch.tensor(Y).to(device)

print(f'Shape of X: {X.shape}, Shape of Y: {Y.shape}')

emb_dim = 32
emb = torch.nn.Embedding(len(stoi), emb_dim)
import torch
import torch.nn as nn

class NextWord(nn.Module):
    def __init__(self, block_size, vocab_size, emb_dim, hidden_size=1024):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lin1 = nn.Linear(block_size * emb_dim, hidden_size)
        self.lin2 = nn.Linear(hidden_size, vocab_size)
    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.shape[0], -1)
        x = torch.tanh(self.lin1(x))
        x = self.lin2(x)
        return x

model = NextWord(block_size, len(stoi), emb_dim, 1024).to(device)
model = torch.compile(model)
model_load_path = '/content/drive/MyDrive/model_emb_32_block_10_tanh.pth'
model.load_state_dict(torch.load(model_load_path, map_location='cpu', weights_only=True))
plot_embeddings(model.emb, itos)

In [ ]:
import torch

block_size = 15
X, Y = [], []

for sentence in cleaned_lines:
    word_list = sentence.split()
    context = [0] * block_size

    for word in word_list + ['.']:
        ix = stoi.get(word, stoi['.'])
        X.append(context.copy())
        Y.append(ix)
        context = context[1:] + [ix]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.tensor(X).to(device)
Y = torch.tensor(Y).to(device)

print(f'Shape of X: {X.shape}, Shape of Y: {Y.shape}')

emb_dim = 32
emb = torch.nn.Embedding(len(stoi), emb_dim)
import torch
import torch.nn as nn

class NextWord(nn.Module):
    def __init__(self, block_size, vocab_size, emb_dim, hidden_size=1024):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lin1 = nn.Linear(block_size * emb_dim, hidden_size)
        self.lin2 = nn.Linear(hidden_size, vocab_size)
    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.shape[0], -1)
        x = torch.tanh(self.lin1(x))
        x = self.lin2(x)
        return x

model = NextWord(block_size, len(stoi), emb_dim, 1024).to(device)
model = torch.compile(model)
model_load_path = '/content/drive/MyDrive/model_emb_32_block_15_tanh.pth'
model.load_state_dict(torch.load(model_load_path, map_location='cpu', weights_only=True))
plot_embeddings(model.emb, itos)

In [ ]:
import torch

block_size = 5
X, Y = [], []

for sentence in cleaned_lines:
    word_list = sentence.split()
    context = [0] * block_size

    for word in word_list + ['.']:
        ix = stoi.get(word, stoi['.'])
        X.append(context.copy())
        Y.append(ix)
        context = context[1:] + [ix]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.tensor(X).to(device)
Y = torch.tensor(Y).to(device)

print(f'Shape of X: {X.shape}, Shape of Y: {Y.shape}')

emb_dim = 64
emb = torch.nn.Embedding(len(stoi), emb_dim)
import torch
import torch.nn as nn

class NextWord(nn.Module):
    def __init__(self, block_size, vocab_size, emb_dim, hidden_size=1024):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lin1 = nn.Linear(block_size * emb_dim, hidden_size)
        self.lin2 = nn.Linear(hidden_size, vocab_size)
    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.shape[0], -1)
        x = torch.relu(self.lin1(x))
        x = self.lin2(x)
        return x

model = NextWord(block_size, len(stoi), emb_dim, 1024).to(device)
model = torch.compile(model)
model_load_path = '/content/drive/MyDrive/model_emb_64_block_5_relu.pth'
model.load_state_dict(torch.load(model_load_path, map_location='cpu', weights_only=True))
plot_embeddings(model.emb, itos)

In [ ]:
import torch

block_size = 10
X, Y = [], []

for sentence in cleaned_lines:
    word_list = sentence.split()
    context = [0] * block_size

    for word in word_list + ['.']:
        ix = stoi.get(word, stoi['.'])
        X.append(context.copy())
        Y.append(ix)
        context = context[1:] + [ix]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.tensor(X).to(device)
Y = torch.tensor(Y).to(device)

print(f'Shape of X: {X.shape}, Shape of Y: {Y.shape}')

emb_dim = 64
emb = torch.nn.Embedding(len(stoi), emb_dim)
import torch
import torch.nn as nn

class NextWord(nn.Module):
    def __init__(self, block_size, vocab_size, emb_dim, hidden_size=1024):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lin1 = nn.Linear(block_size * emb_dim, hidden_size)
        self.lin2 = nn.Linear(hidden_size, vocab_size)
    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.shape[0], -1)
        x = torch.relu(self.lin1(x))
        x = self.lin2(x)
        return x

model = NextWord(block_size, len(stoi), emb_dim, 1024).to(device)
model = torch.compile(model)
model_load_path = '/content/drive/MyDrive/model_emb_64_block_10_relu.pth'
model.load_state_dict(torch.load(model_load_path, map_location='cpu', weights_only=True))
plot_embeddings(model.emb, itos)

In [ ]:
import torch

block_size = 15
X, Y = [], []

for sentence in cleaned_lines:
    word_list = sentence.split()
    context = [0] * block_size

    for word in word_list + ['.']:
        ix = stoi.get(word, stoi['.'])
        X.append(context.copy())
        Y.append(ix)
        context = context[1:] + [ix]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.tensor(X).to(device)
Y = torch.tensor(Y).to(device)

print(f'Shape of X: {X.shape}, Shape of Y: {Y.shape}')

emb_dim = 64
emb = torch.nn.Embedding(len(stoi), emb_dim)
import torch
import torch.nn as nn

class NextWord(nn.Module):
    def __init__(self, block_size, vocab_size, emb_dim, hidden_size=1024):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lin1 = nn.Linear(block_size * emb_dim, hidden_size)
        self.lin2 = nn.Linear(hidden_size, vocab_size)
    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.shape[0], -1)
        x = torch.relu(self.lin1(x))
        x = self.lin2(x)
        return x

model = NextWord(block_size, len(stoi), emb_dim, 1024).to(device)
model = torch.compile(model)
model_load_path = '/content/drive/MyDrive/model_emb_64_block_15_relu.pth'
model.load_state_dict(torch.load(model_load_path, map_location='cpu', weights_only=True))
plot_embeddings(model.emb, itos)

In [ ]:
import torch

block_size = 5
X, Y = [], []

for sentence in cleaned_lines:
    word_list = sentence.split()
    context = [0] * block_size

    for word in word_list + ['.']:
        ix = stoi.get(word, stoi['.'])
        X.append(context.copy())
        Y.append(ix)
        context = context[1:] + [ix]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.tensor(X).to(device)
Y = torch.tensor(Y).to(device)

print(f'Shape of X: {X.shape}, Shape of Y: {Y.shape}')

emb_dim = 64
emb = torch.nn.Embedding(len(stoi), emb_dim)
import torch
import torch.nn as nn

class NextWord(nn.Module):
    def __init__(self, block_size, vocab_size, emb_dim, hidden_size=1024):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lin1 = nn.Linear(block_size * emb_dim, hidden_size)
        self.lin2 = nn.Linear(hidden_size, vocab_size)
    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.shape[0], -1)
        x = torch.tanh(self.lin1(x))
        x = self.lin2(x)
        return x

model = NextWord(block_size, len(stoi), emb_dim, 1024).to(device)
model = torch.compile(model)
model_load_path = '/content/drive/MyDrive/model_emb_64_block_5_tanh.pth'
model.load_state_dict(torch.load(model_load_path, map_location='cpu', weights_only=True))
plot_embeddings(model.emb, itos)

In [ ]:
import torch

block_size = 10
X, Y = [], []

for sentence in cleaned_lines:
    word_list = sentence.split()
    context = [0] * block_size

    for word in word_list + ['.']:
        ix = stoi.get(word, stoi['.'])
        X.append(context.copy())
        Y.append(ix)
        context = context[1:] + [ix]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.tensor(X).to(device)
Y = torch.tensor(Y).to(device)

print(f'Shape of X: {X.shape}, Shape of Y: {Y.shape}')

emb_dim = 64
emb = torch.nn.Embedding(len(stoi), emb_dim)
import torch
import torch.nn as nn

class NextWord(nn.Module):
    def __init__(self, block_size, vocab_size, emb_dim, hidden_size=1024):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lin1 = nn.Linear(block_size * emb_dim, hidden_size)
        self.lin2 = nn.Linear(hidden_size, vocab_size)
    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.shape[0], -1)
        x = torch.tanh(self.lin1(x))
        x = self.lin2(x)
        return x

model = NextWord(block_size, len(stoi), emb_dim, 1024).to(device)
model = torch.compile(model)
model_load_path = '/content/drive/MyDrive/model_emb_64_block_10_tanh.pth'
model.load_state_dict(torch.load(model_load_path, map_location='cpu', weights_only=True))
plot_embeddings(model.emb, itos)

In [ ]:
import torch

block_size = 15
X, Y = [], []

for sentence in cleaned_lines:
    word_list = sentence.split()
    context = [0] * block_size

    for word in word_list + ['.']:
        ix = stoi.get(word, stoi['.'])
        X.append(context.copy())
        Y.append(ix)
        context = context[1:] + [ix]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.tensor(X).to(device)
Y = torch.tensor(Y).to(device)

print(f'Shape of X: {X.shape}, Shape of Y: {Y.shape}')

emb_dim = 64
emb = torch.nn.Embedding(len(stoi), emb_dim)
import torch
import torch.nn as nn

class NextWord(nn.Module):
    def __init__(self, block_size, vocab_size, emb_dim, hidden_size=1024):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lin1 = nn.Linear(block_size * emb_dim, hidden_size)
        self.lin2 = nn.Linear(hidden_size, vocab_size)
    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.shape[0], -1)
        x = torch.tanh(self.lin1(x))
        x = self.lin2(x)
        return x

model = NextWord(block_size, len(stoi), emb_dim, 1024).to(device)
model = torch.compile(model)
model_load_path = '/content/drive/MyDrive/model_emb_64_block_15_tanh.pth'
model.load_state_dict(torch.load(model_load_path, map_location='cpu', weights_only=True))
plot_embeddings(model.emb, itos)